In [40]:
import wptools, wikipedia, re

def checkAllAlpha(test_str):
    for c in test_str:
        if not c.isalpha():
            return False
    return True

def removeBracketsData(test_str):
    ret = ''
    skip1c = 0
    skip2c = 0
    skip3c = 0
    skip4c = 0
    for i in test_str:
        if i == '[':
            skip1c += 1
        elif i == '(':
            skip2c += 1
        elif i == '{':
            skip3c += 1
        elif i == '<':
            skip4c += 1
        elif i == ']' and skip1c > 0:
            skip1c -= 1
        elif i == ')'and skip2c > 0:
            skip2c -= 1
        elif i == '}'and skip2c > 0:
            skip3c -= 1
        elif i == '>'and skip2c > 0:
            skip4c -= 1
        elif skip1c == 0 and skip2c == 0 and skip3c == 0 and skip4c == 0 and i!=',':
            ret += i
    return ret.strip()

def removeHTML(test_str):
    ret = ''
    skip4c = 0
    for i in test_str:
        if i == '<':
            skip4c += 1
        elif i == '>' and skip4c > 0:
            skip4c -= 1
        elif skip4c == 0:
            ret += i
    return ret.strip()

def populate_data(company_name, company_data_dict):
    # company_data_dict = wptools.page(wikipedia.search(company_name)[0]).get_parse(show=False).data['infobox']
    # return company_data_dict
    comp_data_to_store = {'name': company_name, 'type': '', 'industry': '', 'revenue': -1, 'operating_income': -1, 'net_income': -1, 'hq_location': '', 'num_employees': -1, 'founded': -1, 'website':'' }
    try:
        if 'name' in company_data_dict.keys():
            comp_data_to_store['name'] = removeHTML(company_data_dict['name'])
    except:
        comp_data_to_store['name'] = company_name
    try:
        if 'type' in company_data_dict.keys():
            str_to_process = company_data_dict['type']
            list_of_tokens = re.findall(r'([^\[\]]*)', str_to_process)
            for token in list_of_tokens:
                if len(token)>0:
                    comp_data_to_store['type'] = token.split('|')[-1]
                    break
    except:
        comp_data_to_store['type'] = ''
    try:
        if 'industry' in company_data_dict.keys():
            str_to_process = company_data_dict['industry']
            list_of_tokens = re.findall(r'([^\[\]]*)', str_to_process)
            for token in list_of_tokens:
                token = token.strip()
                if len(token)>0 and checkAllAlpha(token[0]):
                    for tok in token.split('|'):
                        if len(tok)>3:
                            comp_data_to_store['industry'] = comp_data_to_store['industry']+ removeBracketsData(tok)+','
            comp_data_to_store['industry'] = comp_data_to_store['industry'][:-1]
    except:
        comp_data_to_store['industry'] = ''
    # print(4)
    try:
        if 'operating_income' in company_data_dict.keys():
            str_to_process = company_data_dict['operating_income']
            index = -1
            operating_income = 0
            if 'million' in str_to_process:
                index = str_to_process.index('million')
                operating_income = 1000000
            elif 'billion' in str_to_process:
                index = str_to_process.index('billion')
                operating_income = 1000000000
            elif 'thousand' in str_to_process:
                index = str_to_process.index('thousand')
                operating_income = 1000
            if operating_income>0:
                num = ''
                l = index-1
                while l>=0 and str_to_process[l]!='$':
                    if str_to_process[l].isdigit() or str_to_process[l]=='.':
                        num = str_to_process[l] + num
                    l-=1
                comp_data_to_store['operating_income'] = float(num) * operating_income
            else:
                comp_data_to_store['operating_income'] = -1.0
    except:
        comp_data_to_store['operating_income'] = -1.0

    try:
        if 'net_income' in company_data_dict.keys() and '$' in company_data_dict['net_income']:
            str_to_process = company_data_dict['net_income']
            index = -1
            net_income = 0
            if 'million' in str_to_process:
                index = str_to_process.index('million')
                net_income = 1000000
            elif 'billion' in str_to_process:
                index = str_to_process.index('billion')
                net_income = 1000000000
            elif 'thousand' in str_to_process:
                index = str_to_process.index('thousand')
                net_income = 1000
            if net_income>0:
                num = ''
                l = index-1
                while l>=0 and str_to_process[l]!='$':
                    if str_to_process[l].isdigit() or str_to_process[l]=='.':
                        num = str_to_process[l] + num
                    l-=1
                comp_data_to_store['net_income'] = float(num) * net_income
            else:
                comp_data_to_store['net_income'] = -1.0
    except:
        comp_data_to_store['net_income'] = -1.0

    try:
        if 'revenue' in company_data_dict.keys():
            str_to_process = company_data_dict['revenue']
            index = -1
            revenue = 0
            if 'million' in str_to_process:
                index = str_to_process.index('million')
                revenue = 1000000
            elif 'billion' in str_to_process:
                index = str_to_process.index('billion')
                revenue = 1000000000
            elif 'thousand' in str_to_process:
                index = str_to_process.index('thousand')
                revenue = 1000
            if revenue>0:
                num = ''
                l = index-1
                while l>=0 and str_to_process[l]!='$':
                    if str_to_process[l].isdigit() or str_to_process[l]=='.':
                        num = str_to_process[l] + num
                    l-=1
                comp_data_to_store['revenue'] = float(num) * revenue
    except:
        comp_data_to_store['revenue'] = -1.0
    # print(5)
    try:
        if 'hq_location' in company_data_dict.keys():
            str_to_process = removeHTML(company_data_dict['hq_location'])
            list_of_tokens = re.findall(r'([^\[\]]*)', str_to_process)
            for token in list_of_tokens:
                token = token.strip()
                if len(token)>0 and token[0].isalpha():
                    for tok in token.split('|'):
                        if len(tok)>0:
                            comp_data_to_store['hq_location'] = comp_data_to_store['hq_location'] + tok + ' '
            comp_data_to_store['hq_location'] = comp_data_to_store['hq_location'].strip()
    except:
        comp_data_to_store['hq_location'] = ''
    # print(6)
    try:
        if 'num_employees' in company_data_dict.keys():
            str_to_process = removeBracketsData(company_data_dict['num_employees'])
            n_emp = 0
            for c in str_to_process:
                if c==',':
                    continue
                elif (not c.isdigit()) and n_emp>0:
                    break
                elif c.isdigit():
                    n_emp = n_emp*10 + int(c)
            comp_data_to_store['num_employees'] = n_emp
    except:
        comp_data_to_store['num_employees'] = 0
    # print(7)
    try:
        if 'founded' in company_data_dict.keys():
            str_to_process = company_data_dict['founded']
            list_of_tokens = re.findall(r'\d{4}', str_to_process)
            for year in list_of_tokens:
                comp_data_to_store['founded'] = int(year)
    except:
        comp_data_to_store['founded'] = -1
    # print(8)
    try:
        if 'website' in company_data_dict.keys():
            str_to_process = company_data_dict['website']
            if '.com' in str_to_process:
                index = str_to_process.index('.com')
                index-=1
                link = ''
                while str_to_process[index]!='.' and str_to_process[index]!='/' and str_to_process[index]!='|':
                    link = str_to_process[index] + link
                    index-=1
                comp_data_to_store['website'] = link+'.com'
                # print(link+'.com')
                # count+=1
            elif '.io' in str_to_process:
                index = str_to_process.index('.io')
                index-=1
                link = ''
                while str_to_process[index]!='.' and str_to_process[index]!='/' and str_to_process[index]!='|':
                    link = str_to_process[index] + link
                    index-=1
                comp_data_to_store['website'] = link+'.io'
                # print(link+'.io')
                # count+=1
            elif '.ai' in str_to_process:
                index = str_to_process.index('.ai')
                index-=1
                link = ''
                while str_to_process[index]!='.' and str_to_process[index]!='/' and str_to_process[index]!='|':
                    link = str_to_process[index] + link
                    index-=1
                comp_data_to_store['website'] = link+'.ai'
                # print(link+'.ai')
                # count+=1
            elif '.org' in str_to_process:
                index = str_to_process.index('.org')
                index-=1
                link = ''
                while str_to_process[index]!='.' and str_to_process[index]!='/' and str_to_process[index]!='|':
                    link = str_to_process[index] + link
                    index-=1
                comp_data_to_store['website'] = link+'.org'
                # print(link+'.org')
                # count+=1
            elif '.us' in str_to_process:
                index = str_to_process.index('.us')
                index-=1
                link = ''
                while str_to_process[index]!='.' and str_to_process[index]!='/' and str_to_process[index]!='|':
                    link = str_to_process[index] + link
                    index-=1
                comp_data_to_store['website'] = link+'.us'
                # print(link+'.us')
                # count+=1
            elif '.net' in str_to_process:
                index = str_to_process.index('.net')
                index-=1
                link = ''
                while str_to_process[index]!='.' and str_to_process[index]!='/' and str_to_process[index]!='|':
                    link = str_to_process[index] + link
                    index-=1
                comp_data_to_store['website'] = link+'.net'
                # print(link+'.net')
                # count+=1
    except:
        comp_data_to_store['website'] = ''

    return comp_data_to_store

In [41]:
import pickle, pandas as pd
pageranksDF = pd.read_csv('pageranks.csv')
domain_pagerank = {}
for i, r in pageranksDF.iterrows():
    try:
        domain_pagerank[r['Domain'].strip()] = float(r['Page Rank Value'])
    except:
        continue

with open('company_url_dict_updated.pkl', 'rb') as f:
    company_url_dict_updated = pickle.load(f)
with open('company_wiki_infobox_data.pkl', 'rb') as f:
    company_wiki_infobox_data = pickle.load(f)
l = []
count = 1
for company_name, comp_data in company_wiki_infobox_data.items():
    data = populate_data(company_name, comp_data)
    if data is None:
        continue
    data['student_careers_url'] = company_url_dict_updated[company_name]
    data['index_name'] = company_name
    if data['website'] in domain_pagerank.keys():
        data['pagerank'] = domain_pagerank[data['website'].strip()]
    else:
        data['pagerank'] = -1
    l.append(data)
    count+=1
count

2051

In [44]:
df = pd.DataFrame(l)
df['revenue'] = df['revenue'].astype('int64', copy=False)
df['net_income'] = df['net_income'].astype('int64', copy=False)
df['operating_income'] = df['operating_income'].astype('int64', copy=False)
df['founded'] = df['founded'].astype('int64', copy=False)
df['num_employees'] = df['num_employees'].astype('int64', copy=False)
df.head()
df.to_csv('companies_wikipedia_data.csv', index=False)
# # import pickle
# with open('wiki_dict.pkl', 'rb') as f:
#     x = pickle.load(f)
# wiki_data_populated = {}
# count = 0
# for k, v in x.items():
#     d = populate_data(k)
#     if d is None :
#         continue
#     wiki_data_populated[k] = d
#     wiki_data_file = open("company_wiki_infobox_data.pkl", "wb")
#     pickle.dump(wiki_data_populated, wiki_data_file)
#     wiki_data_file.close()
#     count+=1
#     if count%100==0:
#         print(count)
# populate_data('Apple (company)')

In [43]:
# import pickle
# import yfinance as yf
# with open('wiki_data_populated.pkl', 'rb') as f:
#     x = pickle.load(f)
# with open('company_url_dict_updated.pkl', 'rb') as f:
#     x2 = pickle.load(f)
# with open('companies_best_match_symbol.pkl', 'rb') as f:
#     companies_best_match_symbol = pickle.load(f)
# import pandas as pd
# row_list = []
# for k, v in x.items():
#     if 'data' in v.keys():
#         continue
#     v['index_name'] = k
#     if v['index_name'] in companies_best_match_symbol.keys():
#         v['stock_symbol'] = companies_best_match_symbol[v['index_name']]
#         try:
#             v['stock_price'] = yf.Ticker(v['stock_symbol']).history(period='1d').tail(1)['Close'].iloc[0]
#         except:
#             v['stock_price'] = 0
#     v['student_careers_url'] = x2[k]
#     row_list.append(v)
# df = pd.DataFrame(row_list)
# df['revenue'] = df['revenue'].astype('int64', copy=False)
# df['founded'] = df['founded'].astype('int64', copy=False)
# df['num_employees'] = df['num_employees'].astype('int64', copy=False)
# df.head()
# # df.to_csv('companies_wikipedia_data.csv', index=False)